# Smart Gift Planner

## Import Libraries

In [32]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

import plotly.express as px

## Import Data

In [8]:
products = pd.read_csv("amazon_products.csv")
categories = pd.read_csv("amazon_categories.csv")

In [9]:
products.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.6,0,365.49,429.99,104,False,300
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.6,0,291.59,354.37,104,False,400
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.5,0,174.99,309.99,104,False,400


In [10]:
categories.head()

,id,category_name
0,1,Beading & Jewelry Making
1,2,Fabric Decorating
2,3,Knitting & Crochet Supplies
3,4,Printmaking Supplies
4,5,Scrapbooking & Stamping Supplies


In [ ]:
# Merge the 2 dfs on category ID
merged = products.merge(
    categories,
    left_on="category_id",
    right_on="id",
    how="left"
)

# Remove numerical category identifiers
merged = merged.drop(columns=["id", "category_id"])

# Save to new .csv
merged.to_csv("merged_products.csv", index=False, encoding="utf-8")

merged.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.50,0,139.99,0.00,False,2000,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.50,0,169.99,209.99,False,1000,Suitcases
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.60,0,365.49,429.99,False,300,Suitcases
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.60,0,291.59,354.37,False,400,Suitcases
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.50,0,174.99,309.99,False,400,Suitcases


In [ ]:
merged.info()
print(merged.isna().sum())
merged.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426337 entries, 0 to 1426336
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   asin               1426337 non-null  object 
 1   title              1426336 non-null  object 
 2   imgUrl             1426337 non-null  object 
 3   productURL         1426337 non-null  object 
 4   stars              1426337 non-null  float64
 5   reviews            1426337 non-null  int64  
 6   price              1426337 non-null  float64
 7   listPrice          1426337 non-null  float64
 8   isBestSeller       1426337 non-null  bool   
 9   boughtInLastMonth  1426337 non-null  int64  
 10  category_name      1426337 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(5)
memory usage: 110.2+ MB
asin                 0
title                1
imgUrl               0
productURL           0
stars                0
reviews              0
price                0
lis

,stars,reviews,price,listPrice,boughtInLastMonth
count,1426337.00,1426337.00,1426337.00,1426337.00,1426337.00
mean,4.00,180.75,43.38,12.45,141.98
std,1.34,1761.45,130.29,46.11,836.27
min,0.00,0.00,0.00,0.00,0.00
25%,4.10,0.00,11.99,0.00,0.00
50%,4.40,0.00,19.95,0.00,0.00
75%,4.60,0.00,35.99,0.00,50.00
max,5.00,346563.00,19731.81,999.99,100000.00


In [18]:
# How many products have reviews?
print((merged['reviews'] != 0).sum())

# How many products are best sellers?
print((merged['isBestSeller'] == True).sum())

295834
8520


In [ ]:
# Drop row with missing product title
merged = merged.dropna(subset=['title'])